In [1]:
import keras
from keras.datasets import cifar10
from keras.layers import Conv2D , MaxPooling2D , Activation ,  Flatten , Dense , Dropout
from keras.models import Sequential 
from keras.preprocessing.image import ImageDataGenerator
from keras.utils import to_categorical
from keras.optimizers import SGD , RMSprop
from keras.callbacks import ModelCheckpoint , EarlyStopping , ReduceLROnPlateau
import numpy as np
import os

batch_size = 32
epochs = 100

(x_train , y_train) , (x_test , y_test) = cifar10.load_data()

print(x_train.shape)
print(x_train.shape[0])
print(x_test.shape[0])



Using TensorFlow backend.


(50000, 32, 32, 3)
50000
10000


In [2]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255



y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(y_train.shape[0])

num_classes = y_train.shape[1]
print(num_classes)

input_shape = x_train[0].shape
print(input_shape)

50000
10
(32, 32, 3)


In [3]:
model = Sequential()

model.add(Conv2D(32 , (3,3) , input_shape = input_shape , activation = 'relu' , padding = 'same'))
model.add(Conv2D(32 , (3,3) , activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Dropout(0.3))
model.add(Conv2D(64 , (3,3) , activation = 'relu' , padding = 'same'))
model.add(Conv2D(64 , (3,3) , activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2,2)))
model.add(Flatten())
model.add(Dense(512 , activation = 'relu' ))
model.add(Dropout(0.5))
model.add(Dense(num_classes , activation = 'relu'))

opt = keras.optimizers.rmsprop(lr = 0.0001 , decay = 1e-6)

model.compile(loss = 'categorical_crossentropy' , optimizer = RMSprop(lr= 0.001) , metrics =['accuracy'])

print(model.summary())

checkpoint = ModelCheckpoint('C:\\Users\\Zahid\\Desktop\\Deep Learning\\',
                            monitor = 'val_loss',
                            mode = 'min',
                            save_best_only = True ,
                            verbose = 1)


reduce_lr  = ReduceLROnPlateau(monitor = 'val_loss' , factor = 0.2 , patience = 3 , verbose = 1 , min_delta = 0.0001)
earlystop = EarlyStopping(monitor = 'val_loss' , 
                         min_delta = 0, #abs value minimum change required before stopping
                        patience = 3, #number of epochs to wait before stopping
                        verbose = 1,
                         restore_best_weights = True)

callbacks = [ checkpoint ,reduce_lr]

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 64)        18496     
_________________________________________________________________

In [ ]:
import pickle


trained_model = model.fit(x_train , y_train , batch_size = batch_size , epochs = 20  , validation_data = (x_test , y_test), shuffle = True , verbose = 1 , callbacks = callbacks)

pickle_out = r'C:\Users\Zahid\Desktop\Deep Learning\image_classifier_cifar10.pickle'
pickle.dump(trained_model.history , pickle_out)
pickle_out.close()
model.save('C:\\Users\\Zahid\\Desktop\\Deep Learning\\cifar10_model.h5')
scores = model.evaluate(x_test , y_test , verbose = 1)
print("test loss" + scores[0])
print('test accuracy' + scores[1])

Train on 50000 samples, validate on 10000 samples
Epoch 1/20
50000/50000 [==============================] - 352s 7ms/step - loss: nan - acc: 0.2585 - val_loss: nan - val_acc: 0.1000

Epoch 00001: val_loss did not improve from 2.14240
Epoch 2/20
   32/50000 [..............................] - ETA: 4:48 - loss: nan - acc: 0.1250

C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:816: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current - self.min_delta, self.best):
C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:709: RuntimeWarning: invalid value encountered in less
  if self.monitor_op(current, self.best):
C:\ProgramData\Anaconda3\lib\site-packages\keras\callbacks.py:1360: RuntimeWarning: invalid value encountered in less
  self.monitor_op = lambda a, b: np.less(a, b - self.min_delta)


40352/50000 [=======================>......] - ETA: 1:03 - loss: nan - acc: 0.1001